# Lab 05: Bedrock Guardrails

## Overview

In this notebook, we add **Bedrock Guardrails** to filter out off-topic, harmful, and sensitive content before it reaches the LLM. This saves tokens on irrelevant requests and improves safety.

**What you'll learn:**
- How to create a Bedrock Guardrail
- How to configure topic, content, and sensitive information filters
- How to integrate guardrails with your agent
- Token savings from blocked queries

**Guardrail Features:**
- **Topic filters**: Block competitor questions, investment advice, medical advice
- **Content filters**: Block violence, hate speech, insults, misconduct
- **Sensitive information (PII)**: Block credit card numbers, SSNs
- **Word filters**: Block specific words and profanity lists
- **Contextual grounding**: Ensure responses are grounded and relevant

## Prerequisites

- Completed Labs 01-04

## Workshop Journey

```
01 Baseline → 02 Quick Wins → 03 Caching → 04 Routing → [05 Guardrails] → 06 Gateway → 07 Evaluations
                                                             ↑
                                                        You are here
```

## Step 1: Setup

In [1]:
from __future__ import annotations

import json
import os
import uuid
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(override=True)

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

region = os.environ.get("AWS_DEFAULT_REGION", "us-east-1")
control_client = boto3.client("bedrock-agentcore-control", region_name=region)
data_client = boto3.client("bedrock-agentcore", region_name=region)
bedrock_client = boto3.client("bedrock", region_name=region)
agentcore_runtime = Runtime()

print(f"Region: {region}")
print(f"Langfuse Host: {os.environ.get('LANGFUSE_BASE_URL', 'https://cloud.langfuse.com')}")

Region: us-east-1
Langfuse Host: https://d2rhlwziq3nnbf.cloudfront.net


## Step 2: Create a Bedrock Guardrail

We'll create a guardrail with multiple filter types:

**Topic Filters (DENY):**
- Questions about competitor products
- Investment/financial advice requests
- Medical advice requests

**Content Filters:**
- Violence, hate speech, insults (HIGH/MEDIUM strength)

**Sensitive Information (PII):**
- BLOCK: Credit card numbers, SSNs (blocks entire request with custom message)

In [2]:
# Create the guardrail
guardrail_name = "customer-support-guardrail"

try:
    guardrail_response = bedrock_client.create_guardrail(
        name=guardrail_name,
        description="Block off-topic, harmful, and sensitive content for customer support agent",
        blockedInputMessaging="I'm unable to process your request. This may be due to sensitive information in your message or a topic outside my scope. Please remove any personal financial data and try again, or ask about TechMart products, returns, or technical support.",
        blockedOutputsMessaging="I apologize, but I cannot provide that type of information. Please let me know if you have questions about TechMart products or services.",
        topicPolicyConfig={
            "topicsConfig": [
                {
                    "name": "competitor-products",
                    "definition": "Questions comparing TechMart to competitors like Apple, Samsung, Dell, HP, or asking which brand is better",
                    "examples": [
                        "Is your laptop better than a MacBook?",
                        "Should I buy from you or Best Buy?",
                        "How do you compare to Apple?",
                    ],
                    "type": "DENY",
                },
                {
                    "name": "investment-advice",
                    "definition": "Questions about financial investments, stock market, cryptocurrency, or financial planning",
                    "examples": [
                        "Should I invest in tech stocks?",
                        "Is now a good time to buy crypto?",
                        "What stocks do you recommend?",
                    ],
                    "type": "DENY",
                },
                {
                    "name": "medical-advice",
                    "definition": "Questions about medical diagnoses, treatments, or health conditions",
                    "examples": [
                        "Can screen time cause headaches?",
                        "Is blue light bad for my eyes?",
                        "What medicine should I take?",
                    ],
                    "type": "DENY",
                },
            ]
        },
        contentPolicyConfig={
            "filtersConfig": [
                {"type": "VIOLENCE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
                {"type": "HATE", "inputStrength": "HIGH", "outputStrength": "HIGH"},
                {"type": "INSULTS", "inputStrength": "MEDIUM", "outputStrength": "MEDIUM"},
            ]
        },
        sensitiveInformationPolicyConfig={
            "piiEntitiesConfig": [
                {"type": "CREDIT_DEBIT_CARD_NUMBER", "action": "BLOCK"},
                {"type": "US_SOCIAL_SECURITY_NUMBER", "action": "BLOCK"},
            ],
        },
    )

    guardrail_id = guardrail_response["guardrailId"]
    print(f"Guardrail created: {guardrail_id}")

except bedrock_client.exceptions.ConflictException:
    # Guardrail already exists, get its ID
    response = bedrock_client.list_guardrails()
    for g in response["guardrails"]:
        if g["name"] == guardrail_name:
            guardrail_id = g["id"]
            print(f"Using existing guardrail: {guardrail_id}")
            break

Using existing guardrail: adrtlvfiqcu8


In [3]:
# Store guardrail ID in SSM Parameter Store
ssm_client = boto3.client("ssm", region_name=region)
ssm_param_name = "/app/customersupport/guardrail_id"

ssm_client.put_parameter(
    Name=ssm_param_name,
    Value=guardrail_id,
    Type="String",
    Overwrite=True,
)

print(f"Guardrail ID: {guardrail_id}")
print(f"Stored in SSM: {ssm_param_name}")

Guardrail ID: adrtlvfiqcu8
Stored in SSM: /app/customersupport/guardrail_id


## Step 3: Review the Guardrails Agent

The v5 agent integrates guardrails by passing `guardrail_id` and `guardrail_version` to the `BedrockModel`. When a query triggers the guardrail, Bedrock intercepts the request **before** the LLM processes it, returning the `blockedInputMessaging` we configured above.

In [4]:
# Review the v5 agent code
agent_file = Path("agents/v5_guardrails.py")
print(agent_file.read_text())

"""
V5 Guardrails Agent - Bedrock Guardrails for content filtering.
- Block off-topic queries before LLM processes them
- Save tokens on irrelevant requests
"""

import os

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from strands.types.content import SystemContentBlock

from utils.agent_config import (
    MODEL_HAIKU,
    MODEL_SONNET,
    SYSTEM_PROMPT_TEXT,
    classify_query_complexity,
    setup_langfuse_telemetry,
)
from utils.tools import get_product_info, get_return_policy, get_technical_support, web_search

setup_langfuse_telemetry()

app = BedrockAgentCoreApp()

GUARDRAIL_ID = os.environ.get("GUARDRAIL_ID")

SYSTEM_PROMPT = [
    SystemContentBlock(text=SYSTEM_PROMPT_TEXT),
    SystemContentBlock(cachePoint={"type": "default"}),
]


@app.entrypoint
def invoke(payload):
    user_input = payload.get("prompt", "")

    telemetry = StrandsTelemetry()
    

## Step 4: Deploy the Guardrails Agent

In [ ]:
agent_name = "customer_support_v5_guardrails"
agent_file = str(Path("agents/v5_guardrails.py").absolute())
requirements_file = str(Path("requirements-for-agentcore.txt").absolute())

print(f"Agent name: {agent_name}")
print(f"Agent file: {agent_file}")
print(f"Requirements: {requirements_file}")

# Clean up any existing build files from previous labs
for f in ["Dockerfile", ".dockerignore", ".bedrock_agentcore.yaml"]:
    p = Path(f)
    if p.exists():
        p.unlink()
        print(f"Removed existing: {f}")

print(f"Configuring agent: {agent_name}")
agentcore_runtime.configure(
    entrypoint=agent_file,
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file=requirements_file,
    region=region,
    agent_name=agent_name,
)

In [6]:
# Modify Dockerfile for Langfuse
import re

dockerfile_path = Path("Dockerfile")
if dockerfile_path.exists():
    content = dockerfile_path.read_text()
    if "opentelemetry-instrument" in content:
        content = re.sub(
            r'CMD \["opentelemetry-instrument", "python", "-m", "([^"]+)"\]', r'CMD ["python", "-m", "\1"]', content
        )
        dockerfile_path.write_text(content)
        print("Dockerfile modified for Langfuse")
    else:
        print("Dockerfile already configured or using different format")

Dockerfile modified for Langfuse


In [7]:
# Read guardrail ID from SSM (stored in Step 2)
ssm_response = ssm_client.get_parameter(Name="/app/customersupport/guardrail_id")
guardrail_id_from_ssm = ssm_response["Parameter"]["Value"]
print(f"Retrieved guardrail ID from SSM: {guardrail_id_from_ssm}")

# Inject as environment variable at deployment time
env_vars = {
    "LANGFUSE_BASE_URL": os.environ.get("LANGFUSE_BASE_URL"),
    "LANGFUSE_PUBLIC_KEY": os.environ.get("LANGFUSE_PUBLIC_KEY"),
    "LANGFUSE_SECRET_KEY": os.environ.get("LANGFUSE_SECRET_KEY"),
    "GUARDRAIL_ID": guardrail_id_from_ssm,
    "PYTHONUNBUFFERED": "1",
}

print("Deploying to AgentCore Runtime...")
launch_result = agentcore_runtime.launch(env_vars=env_vars, auto_update_on_conflict=True)
agent_arn = launch_result.agent_arn
print(f"Agent deployed: {agent_arn}")

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'customer_support_v5_guardrails' to account 739907928487 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_v5_guardrails


Retrieved guardrail ID from SSM: adrtlvfiqcu8
Deploying to AgentCore Runtime...


ECR repository available: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v5_guardrails
Getting or creating execution role for agent: customer_support_v5_guardrails
Using AWS region: us-east-1, account ID: 739907928487
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-c681520de5


✅ Reusing existing ECR repository: 739907928487.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-customer_support_v5_guardrails


✅ Reusing existing execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-c681520de5
Execution role available: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-c681520de5
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: customer_support_v5_guardrails
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-c681520de5
Reusing existing CodeBuild execution role: arn:aws:iam::739907928487:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-c681520de5
Using dockerignore.template with 46 patterns for zip filtering
Uploaded source to S3: customer_support_v5_guardrails/source.zip
Updated CodeBuild project: bedrock-agentcore-customer_support_v5_guardrails-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.3s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 9.0s
🔄 DO

Agent deployed: arn:aws:bedrock-agentcore:us-east-1:739907928487:runtime/customer_support_v5_guardrails-1PXVhe426m


In [ ]:
# Save the agent ARN for later use
agent_arn = launch_result.agent_arn
print(f"Agent ARN: {agent_arn}")

## Step 5: Test Guardrails

We'll send five test queries to demonstrate guardrail behavior:

| Test | Query Type | Expected Result |
|------|------------|-----------------|
| 1 | Competitor comparison | **BLOCKED** - Topic filter triggers |
| 2 | Investment advice | **BLOCKED** - Topic filter triggers |
| 3 | Medical advice | **BLOCKED** - Topic filter triggers |
| 4 | Credit card number | **BLOCKED** - PII filter triggers |
| 5 | Valid support question | **PASSED** - Full response |

**What to observe:**
- Blocked queries return the `blockedInputMessaging` we configured
- Blocked queries use minimal tokens (just the user query)
- Valid queries use the full token budget (system prompt + tools + user query)

In [8]:
def invoke_agent(prompt):
    """Invoke the agent via AgentCore API."""
    response = data_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=json.dumps({"prompt": prompt}).encode(),
    )
    return json.loads(response["response"].read().decode("utf-8"))

In [9]:
# Import Langfuse metrics helper
from utils.langfuse_metrics import (
    clear_metrics,
    collect_metric,
    get_latest_trace_metrics,
    print_metrics,
    print_metrics_table,
)

# Clear any previously collected metrics
clear_metrics()
print("Metrics helper ready")

Metrics helper ready


In [10]:
# Test prompts designed to demonstrate guardrail filtering
TEST_PROMPTS = [
    # Topic filter: Competitor comparison (should be BLOCKED)
    ("Competitor Question", "Is your laptop better than a MacBook Pro?"),
    # Topic filter: Investment advice (should be BLOCKED)
    ("Investment Advice", "Should I invest in tech stocks right now?"),
    # Topic filter: Medical advice (should be BLOCKED)
    ("Medical Advice", "Is blue light from screens bad for my eyes?"),
    # PII test: Credit card in query (should be BLOCKED)
    ("PII - Credit Card", "I want to pay with my card 4111-1111-1111-1111, can you process my order?"),
    # Valid query (should PASS) - for comparison
    ("Valid Query", "What is your return policy for laptops?"),
]

# Run all tests and observe guardrail behavior
print("=" * 70)
print("GUARDRAIL DEMONSTRATION")
print("=" * 70)
print("Testing various query types to show guardrail filtering in action.")
print("- BLOCKED: Query rejected before reaching LLM (saves 100% tokens)")
print("- PASSED: Valid query processed normally")
print("=" * 70)

for test_name, prompt in TEST_PROMPTS:
    print("\n" + "-" * 70)
    print(f"Test: {test_name}")
    print(f"Query: {prompt}")
    print("-" * 70)

    response = invoke_agent(prompt)
    print(f"Response: {response}")

    # Fetch and collect metrics
    metrics = get_latest_trace_metrics(
        agent_name="customer-support-v5-guardrails",
        wait_seconds=5,
        max_retries=5,
        timeout_seconds=120,
    )
    print_metrics(metrics, test_name)
    collect_metric(metrics, test_name)

GUARDRAIL DEMONSTRATION
Testing various query types to show guardrail filtering in action.
- BLOCKED: Query rejected before reaching LLM (saves 100% tokens)
- PASSED: Valid query processed normally

----------------------------------------------------------------------
Test: Competitor Question
Query: Is your laptop better than a MacBook Pro?
----------------------------------------------------------------------
Response: {'response': "I'm sorry, but this question is outside my scope as a TechMart customer support assistant. I can help you with product information, returns, and technical support.", 'guardrails_enabled': True}

                    LANGFUSE METRICS
  Test:          Competitor Question
  Trace ID:      43e04734c7871ea161486bb54a4864aa
------------------------------------------------------------
  Latency:       0.66s
  Cost:          $0.000885
  Input tokens:  25
  Output tokens: 54
  Total tokens:  79
------------------------------------------------------------
  Cache r

In [ ]:
# Print summary table
print_metrics_table()

# Save metrics for comparison in later notebooks
from utils.langfuse_metrics import save_metrics
save_metrics("v5")

## Step 6: Analyze Results

Look at the metrics table above. Notice the difference between blocked and valid queries:

**Key Observations:**

1. **Token Difference**: Blocked queries use significantly fewer input tokens than valid queries. The guardrail intercepts the request before the full system prompt and tools are sent to the LLM.

2. **Faster Response**: Blocked queries return much faster since no LLM inference is needed - the guardrail returns the blocked message immediately.

3. **Consistent Messaging**: All blocked queries return the same professional message we configured in `blockedInputMessaging`, providing a consistent user experience.

4. **Different Block Reasons**: Both topic violations (competitor, investment, medical) and PII (credit card) trigger the same blocking behavior.

> **Note on Pricing**: Bedrock Guardrails has its own pricing based on text units processed. See [Bedrock Guardrails pricing](https://aws.amazon.com/bedrock/pricing/) for details. The value of guardrails is primarily in **safety, compliance, and user experience** rather than cost optimization.

In [ ]:
# View detailed traces in Langfuse
langfuse_base_url = os.environ.get("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")
print(f"View detailed traces at: {langfuse_base_url}")
print("\nFilter by tags: 'guardrails' to see all v5 agent traces")

## Summary

In this notebook, we added **Bedrock Guardrails** to intercept and block inappropriate requests before they reach the LLM.

**What We Configured:**
- **Topic filters**: Block competitor comparisons, investment advice, and medical advice
- **Content filters**: Block violent, hateful, and inappropriate content  
- **PII filters**: Block credit card numbers and SSNs

**What We Observed:**
- Blocked queries return immediately with a consistent, professional message
- The guardrail intercepts requests before the full context is sent to the LLM
- Both topic violations and PII triggers result in the same blocking behavior

**Benefits of Guardrails:**
- **Safety**: Prevent the agent from engaging with harmful or inappropriate content
- **Compliance**: Automatically block sensitive data like credit cards and SSNs
- **Consistency**: Provide uniform messaging for out-of-scope requests
- **Scope Control**: Keep the agent focused on its intended purpose

**Next notebook:** [06-agentcore-gateway.ipynb](./06-agentcore-gateway.ipynb) - Add semantic tool search with AgentCore Gateway

## Cleanup (Optional)

In [ ]:
# Uncomment to delete resources created in this lab
# agentcore_runtime.destroy(delete_ecr_repo=True)
# print(f"Deleted agent and ECR repository: {agent_name}")

# # Delete the guardrail
# bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)
# print(f"Deleted guardrail: {guardrail_id}")